# Downloading required Frameworks

In [ ]:
! pip install -q --upgrade google-generativeai langchain-google-genai langchain_pinecone langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
pip install pypdf langchain PyPDF2 langchain_google_genai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


Importing Libraries

# Importing Libraries

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from google.colab import files
import pypdf
from pinecone import Pinecone

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

To get keys using google colab userdata

In [ ]:
from google.colab import userdata

Two API secret keys will be needed from Google Gemini Pro for creating embeddings and Pinecone for storing those embeddings

In [ ]:
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import os

pinecone_api_key = userdata.get("PINECONE_API_KEY")

import time

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

# Loading the document

To upload files required for context of QA chatbot uploading the file into the contents of Google colab.

In [ ]:
uploaded_files = files.upload()
doc = [open(name, "rb") for name in uploaded_files.keys()]

Saving Attention.pdf to Attention.pdf


Directory which will consist all the documents related to context on which Chatbot will give answers

In [ ]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [ ]:
doc=read_doc('/content/')
len(doc) #No. of pages in the document directory

15

In [ ]:
doc

[Document(metadata={'source': '/content/Attention.pdf', 'page': 0}, page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Trans

# Chunking

Chunking using Recursive text splitter to overcome the issue of overlapping words

In [ ]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [ ]:
documents=chunk_data(docs=doc)
len(documents)

15

Google Gemini Pro is used since it's API is free to use

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7bb6edebe950>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

# Pinecone Indexing

The pinecone key has to be fed into the environment since the same key will be used by vector store and pinecone indexer

In [ ]:
import os
os.environ['PINECONE_API_KEY'] = userdata.get("PINECONE_API_KEY")

In [ ]:
pinecone_api_key=os.environ['PINECONE_API_KEY']

In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=pinecone_api_key)

To set up the indexer which can be also modified with the help of Pinecone UI where different metrics and servers can be chosen. The dimensions of the LLM Gemini Pro vector embeddings are 768 (1536 for Open AI API). The metric used here is cosine and cloud server can be set which will be accessed through Pinecone.

In [ ]:
index_name = "langchain-rag-index"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)

# Pinecone Vectorstore

In [ ]:
from langchain_pinecone import PineconeVectorStore


In [ ]:
docsearch = PineconeVectorStore.from_documents(documents, embeddings, index_name=index_name)

In [ ]:
def retrieve_query(query,k=2):
    matching_results=docsearch.similarity_search(query,k=k)
    return matching_results

# Conversational chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain

One shot prompting is used to augment the given use case since it is a qa chatbot a qa chain is used for effective answers with respect to questions based on context

In [ ]:
prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details in 200 words. If the answer is not in
    the provided context, just say, ""answer is not available in the context," don't provide the wrong answer.\n\n
    Context:\n{context}\n
    Question:\n{question}\n
    Answer:
    """
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3,google_api_key=GOOGLE_API_KEY)
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [ ]:
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "What is Attention"
answer = retrieve_answers(our_query)

[Document(metadata={'page': 3.0, 'source': '/content/Attention.pdf'}, page_content='Scaled Dot-Product Attention\n Multi-Head Attention\nFigure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several\nattention layers running in parallel.\nof the values, where the weight assigned to each value is computed by a compatibility function of the\nquery with the corresponding key.\n3.2.1 Scaled Dot-Product Attention\nWe call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of\nqueries and keys of dimension dk, and values of dimension dv. We compute the dot products of the\nquery with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the\nvalues.\nIn practice, we compute the attention function on a set of queries simultaneously, packed together\ninto a matrix Q. The keys and values are also packed together into matrices KandV. We compute\nthe matrix of outputs as:\nAttention( Q, K, V ) = sof

In [ ]:
answer

'Attention is a mechanism that allows a neural network to focus on specific parts of its input. In the context of natural language processing, attention allows a model to focus on specific words or phrases in a sentence. This can be useful for tasks such as machine translation, where the model needs to understand the meaning of a sentence in order to translate it accurately.\n\nAttention is typically implemented using a function that takes a query and a set of keys and values as input. The query is typically a representation of the part of the input that the model is interested in focusing on. The keys are typically representations of the different parts of the input, and the values are typically representations of the information associated with each part of the input. The attention function then computes a set of weights that are used to combine the values, resulting in a single output vector.\n\nThe weights are computed using a compatibility function, which measures the similarity b